# Derivadas superiores

Hasta ahora hemos visto que, usando diferenciación automática, podemos calcular la derivada de funciones de una variable esencialmente con un error del orden del epsilon de la máquina.

La pregunta que abordaremos ahora, es cómo hacer para calcular la segunda derivada, y derivadas de orden superior.

Una posibilidad, específica para el caso de la segunda derivada, es proceder como en el caso anterior, es decir, definir una *terna ordenada* donde la primer componente es el valor de la función en $x_0$, i.e., $f(x_0)$, el de la segunda es el valor de la primer derivada $f'(x_0)$, y la tercer componente tiene el valor de la segunda derivada, $f^{(2)}(x_0) = f''(x_0)$. 


Con esta definición, las operaciones aritméticas vienen dadas por:

\begin{eqnarray}
\vec{u} + \vec{v} & = & (u + v, \quad u'+ v', \quad u''+v''),\\
\vec{u} - \vec{v} & = & (u - v, \quad u'- v', \quad u''-v''),\\
\vec{u} \times \vec{v} & = & (u v, \quad u v' + u' v, \quad u v'' + 2 u' v' + u'' v),\\
\frac{\vec{u}}{\vec{v}} & = & \Big( \frac{u}{v}, \quad \frac{u'-( u/v) v'}{v}, \quad 
\frac{u'' - 2 (u/v)' v' - (u/v)v'' }{v}\Big).\\
\end{eqnarray}

Claramente, este proceso es muy ineficiente para derivadas de orden aún más alto, dado que las expresiones se complican y es fácil cometer errores.

# Series de Taylor

El punto importante a recordar, es que las derivadas de orden superior de una función $f(x)$ en un punto $x_0$ están contenidas en el desarrollo de Taylor de la función entorno a ese punto. La suposición importante en esto es que $f(x)$ es suficientemente suave; por simplicidad supondremos que $f(x)$ es ${\cal C}^\infty$ y que estamos suficientemente cerca del punto $x_0$, i.e., $|x-x_0|\ll 1$. 


La serie de Taylor de $f(x)$ viene dada por

\begin{eqnarray}
f(x) & = & f(x_0) + f^{(1)}(x_0) (x-x_0) + \frac{f^{(2)}(x_0)}{2!} (x-x_0)^2 + \dots + \frac{f^{(k)}(x_0)}{k!} (x-x_0)^k + \dots,\\
& = & f_{[0]} + f_{[1]} (x-x_0) + f_{[2]} (x-x_0)^2 + \dots + f_{[k]} (x-x_0)^k + \dots,\\
\end{eqnarray}

donde los coeficientes *normalizados* de Taylor $f_{[k]}$ que aparecen en la segunda línea de la ecuación anterior se definen como

\begin{equation}
f_{[k]} = \frac{f^{(k)}(x_0)}{k!} = \frac{1}{k!}\frac{{\rm d}^k f}{{\rm d}x^k}(x_0).
\end{equation}



Vale la pena **enfatizar** que la expresión anterior es exacta en tanto que la serie **no** sea truncada. En el caso de que la serie truncada a orden k, el [teorema de Taylor](https://en.wikipedia.org/wiki/Taylor%27s_theorem) asegura que el residuo (error del truncamiento) se puede escribir como:

\begin{equation}
{\cal R_{k}} = \frac{f^{(k+1)}\,(\xi)}{(k+1)!} (x-x_0)^{k+1},
\end{equation}

donde $\xi$ es un punto que pertenece al interval $[x_0,x]$.


Si la serie es truncada, la aproximación es un polinomio de orden $k$ (grado máximo es $k$) en $x$. Dado que los polinomios en una variable están definidos por $k+1$ coeficientes, entonces pueden ser mapeados a vectores en $\mathbb{R}^{k+1}$. 

Las operaciones aritméticas, en este caso, vienen dadas por:

\begin{eqnarray}
(f+g)_{[k]} & = & f_{[k]} + g_{[k]} ,\\
(f-g)_{[k]} & = & f_{[k]} - g_{[k]} ,\\
(f \cdot g)_{[k]} & = & \sum_{i=0}^k f_{[i]} \,g_{[k-i]} \, ,\\
\Big(\frac{f}{g}\Big)_{[k]} & = & \frac{1}{g_{[0]}}
\Big( f_{[k]} - \sum_{i=0}^{k-1} \big(\frac{f}{g}\big)_{[i]} \, g_{[k-i]} \Big) . \\
\end{eqnarray}

### Ejercicio

Implementen una nueva estructura paramétrica (`type`) que defina el tipo `Taylor`, donde el parámetro debe ser un subtipo de `Number`. Definan métodos que implementen las operaciones aritméticas básicas (`+`, `-`, `*`, `/`) y la igualdad (`==`). Esto deberá ser incluido en un módulo.

Incluyan pruebas (en el archivo "runtests.jl") para cada uno de los métodos que implementen.


In [1]:
include("Taylor.jl")

TS

In [2]:
"""
Taylor

Es un tipo parametrizado por subtipos de Number. Consiste en una entrada
vecotrial Taylor.taylor_vec donde se mapean los primeros k coeficientes
de Taylor. La k escogida en esta estructura en particular es 10 y está
implementada para que si se mete un arreglo con menos de 10 entradas se
complemente con ceros restantes y si se mete un arreglo con más de 10 
entradas se tomen sólo las primeras 10.
"""
type Taylor{T<:Number}
    # Aquí se declara taylor_vec que es un Array unidimensional de tipo T.
    taylor_vec::Array{T, 1}
    
    # Aquí hay un constructor interno que lo que va a hacer es tomar el
    # arreglo taylor_vec y ver si tiene un tamaño mayor o menor que la
    # k que es donde se trunca la serie (en este caso k = 10).
    function Taylor(taylor_vec::Array{T, 1})
        if size(taylor_vec, 1) < 10
            # Si hay menos de k entradas, se utiliza vcat para meter ceros 
            # restantes en taylor_vec y luego se reconstruye.
            taylor_vec = vcat(taylor_vec, zeros(T, 10 - size(taylor_vec,1)))
            new(taylor_vec)
            else
            # En caso contrario se seleccionan las primeras k entradas.
            taylor_vec = taylor_vec[1:10]
            new(taylor_vec)
        end
    end
end

# Este es un constructor externo para evitar problemas de conversión. Véase:
# http://docs.julialang.org/en/release-0.4/manual/constructors/
# Específicamente la sección de Parametric Constructors.
Taylor{T<:Number}(taylor_vec::Array{T, 1}) = Taylor{T}(taylor_vec)

# Adicionalmente se declara un método para tener Taylor para un escalar.
Taylor{T<:Number}(a::T) = Taylor(a*ones(T,1))

Taylor{T<:Number}

Por ejemplo, si ponemos la serie con 15 entradas se ve que se corta hasta 10 coeficientes:

In [3]:
Taylor(ones(Int, 15))

Taylor{Int64}([1,1,1,1,1,1,1,1,1,1])

En cambio, si metemos un número insuficiente también se arregla:

In [4]:
Taylor(ones(Float64, 4))

Taylor{Float64}([1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0])

Finalmente, la implementación escalar funciona adecuandamente:

In [5]:
Taylor(2)

Taylor{Int64}([2,0,0,0,0,0,0,0,0,0])

In [6]:
import Base: +, -, *, /, ==

# La suma entre los dos Taylors y entre un escalar.
+(a::Taylor, b::Taylor) = Taylor(a.taylor_vec + b.taylor_vec)
+(a::Taylor, b::Number) = a + Taylor(b)
+(a::Number, b::Taylor) = Taylor(a) + b
# La resta se implementa igualmente.
-(a::Taylor, b::Taylor) = Taylor(a.taylor_vec - b.taylor_vec)
-(a::Taylor, b::Number) = a - Taylor(b)
-(a::Number, b::Taylor) = Taylor(a) - b
# Los operadores unitarios y la comparación.
+(a::Taylor) = a
-(a::Taylor) = Taylor(-a.taylor_vec)
==(a::Taylor, b::Taylor) = a.taylor_vec == b.taylor_vec
# La mutiplicación se implementa usando dos for's. El primero que va
# desde 1, size(a.taylor_vec, 1) construye el arreglo entero de k 
# elementos. El segundo for anidado es la suma que se da en la teoría
# de arriba. Sólo hay que tener cuidado porque los índices empiezan en
# uno y no en cero como en la teoría.
*(a::Taylor, b::Taylor) = Taylor([sum([a.taylor_vec[i]*b.taylor_vec[k-i+1] 
        for i in range(1,k)]) for k in range(1,size(a.taylor_vec,1))])
*(a::Taylor, b::Number) = a*Taylor(b)
*(a::Number, b::Taylor) = Taylor(a)*b

# La división resulta ser un poco más complicada así que se opta por
# definirla en una función explícita y luego meterla en el método de /.
function /(a::Taylor, b::Taylor)
    # Lo primero que se hace es construir el arreglo de la división que
    # sea del mismo tamaño de k elementos y lleno de ceros enteros.
    # Enteros para que se puedan promover en el caso de que los arreglos
    # sean de tipo Float, Complex, etc.
    div_vec = 0*a.taylor_vec
    # El primer elemento es sencillamente:
    div_vec[1] = a.taylor_vec[1]/b.taylor_vec[1]
    # De esta manera nos evitamos problemas con la suma que iría sobre un
    # rango 1 a 0.
    # Ahora podemos llenar los elementos que falta. Nuestro rango empieza
    # en 2 pero es de longitud(div_vec,1) -1:
    for k in range(2, size(div_vec,1)-1)
        # sd será la suma que aparece en la ecuación, inicializada como cero.
        sd = 0
        # Con el siguiente for se ejecuta la suma utilizando los coeficientes
        # de div_vec que ya se generaron.
        for i in range(1,k-1)
            sd = sd + (div_vec[i])*b.taylor_vec[k-i+1]
        end
        div_vec[k] = (1/b.taylor_vec[1])*(a.taylor_vec[k]-sd)
    end
    return Taylor(div_vec)
end       
/(a::Taylor, b::Number) = a/Taylor(b)
/(a::Number, b::Taylor) = Taylor(a)/b

/ (generic function with 54 methods)

Probemos con un par de ejemplos si $a(x) = 1 + x + x^2 + x^3$ y $b(x) = 1+x$ esperamos que la división se $1+x^2$:

In [7]:
a = Taylor([1,1])
b = Taylor([1,1,1,1])
b/a

Taylor{Int64}([1,0,1,0,0,0,0,0,0,0])

In [8]:
# Muestren que su código funciona con tests adecuados; para los detalles ver 
# http://julia.readthedocs.org/en/release-0.4/stdlib/test/
using Base.Test


# Funciones de polinomios

El siguiente punto, es cómo definir funciones de polinomios. 

Como veremos aquí, esto se basará en plantear una ecuación diferencial apropiada, cuya solución es, precisamente, la expresión que estamos buscando. Este punto es *importante*, y muestra que hay una conexión importante con la solución de ecuaciones diferenciales.

Como ejemplo consideraremos la función

\begin{equation}
E(x) = \exp\big(g(x)\big),
\end{equation}

donde 

\begin{equation}
g(x) = \sum_{k=0}^\infty g_{[k]} (x-x_0)^k
\end{equation}

está escrita como una serie de Taylor (¡exacta!) alrededor de $x_0$. 


El primer punto, es que escribiremos a $E(x)$ como una serie de Taylor alrededor de $x_0$, es decir,

\begin{equation}
E(x) = \sum_{k=0}^\infty E_{[k]} (x-x_0)^k.
\end{equation}

El objetivo es determinar $E_{[k]}$ para *toda* $k$.

Dado que $E(x)$ esun polinomio en $x$, su derivada viene dada por

\begin{equation}
\frac{{\rm d} E(x)}{{\rm d}x} = \sum_{k=1}^\infty k E_{[k]}\, (x-x_0)^{k-1} .
\end{equation}

Por otra parte, la derivada de $E(x)$ en términos de $g(x)$ está dada por

\begin{equation}
\frac{{\rm d} E(x)}{{\rm d}x} = \exp\big(g(x)\big) \frac{{\rm d} g(x)}{{\rm d}x} = E(x) \frac{{\rm d} g(x)}{{\rm d}x},
\end{equation}

donde del lado derecho aparece la derivada de $g(x)$. Ya que $g(x)$ *también* está escrita en forma polinomial, su derivada es

\begin{equation}
\frac{{\rm d} g(x)}{{\rm d}x} = \sum_{k=1}^\infty k g_{[k]}\, (x-x_0)^{k-1} .
\end{equation}


Tenemos, entonces, todo lo que requerimos para escribir el lado derecho de la ecuación diferencial y explotar la aritmética de polinomios. 

\begin{eqnarray}
E(x) \frac{{\rm d} g(x)}{{\rm d}x}& = & 
\Big[ \sum_{k=0}^\infty E_{[k]} (x-x_0)^k \Big]
\Big[ \sum_{j=1}^\infty j g_{[j]} (x-x_0)^{j-1}\Big] \\
 & = & \sum_{k=1}^\infty \Big[ \sum_{j=0}^k j g_{[j]} E_{[k-j]} \; \Big] (x-x_0)^{k-1} .\\
\end{eqnarray}

La segunda línea se obtiene reordenando los términos al fijar la potencia de $(x-x_0)$, esto es, $k+j$ se toma como un nuevo índice ($k$), y el nuevo índice $j$ describe el índice del producto de los polinomios. (La potencia se deja de la forma $k-1$ ya que el lado izquierdo de la ecuación aparece así.)

Igualando con el lado izquierdo de la ecuación diferencial, que sólo involucra a la derivada de $E(x)$, tenemos que se debe cumplir

\begin{equation}
E_{[k]} = \frac{1}{k} \sum_{j=0}^k j g_{[j]} \, E_{[k-j]} = 
\frac{1}{k} \sum_{j=0}^{k} (k-j) g_{[k-j]} \, E_{[j]}, \qquad k=1,2,\dots,
\end{equation}

incluyendo *la condición inicial*

\begin{equation}
E_{[0]} = \exp\big(g(x_0)\big).
\end{equation}

Estas relaciones *de recurrencia* permiten calcular $\exp\big(g(x)\big)$, para cualquier polinomio $g(x)$.

Para el caso concreto $g(x) = x$ alrededor de $x_0=0$, donde tenemos $g_{[j]} = \delta_{j,1}$, obtenemos

\begin{eqnarray}
E_{[0]} & = & 1,\\
E_{[k]} & = & \frac{1}{k} E_{[k-1]} = \frac{1}{k(k-1)} E_{[k-2]} = \dots = \frac{1}{k!} E_{[0]} = \frac{1}{k!}\ ,
\end{eqnarray}

que es el resultado bien conocido.

### Ejercicio

Obtengan las relaciones de recurrencia para las funciones $L(x) = \log\big(g(x)\big)$, $P_\alpha(x) = \big(g(x)\big)^\alpha$, $S(x) = \sin\big(g(x)\big)$, $C(x) = \cos\big(g(x)\big)$ usando el mismo procedimiento que arriba. Implementen métodos adecuados para estas funciones en el módulo, actuando sobre estructuras `Taylor` e incluyan pruebas.

**NOTA** Los ejercicios de este notebook constituyen el contenido de la Tarea6.

Primero empezaremos implementando la exponencial como se vio arriba:

In [9]:
import Base: exp
# Esta es la función exponencial aplicada a estructuras Taylor.
function exp(a::Taylor)
    # Este primer if está armado para que no se regrese un error de
    # inexactitud de meter un entero a una exponencial que trabaja 
    # mínimo con números flotantes.
    if eltype(a.taylor_vec) <: Integer
        a = Taylor(convert(Array{Float64,1}, a.taylor_vec))
    end
    # Inicializamos el arreglo e_vec que será quien llene la 
    # estructura Taylor asociada a la exponencial:
    e_vec = 0.*a.taylor_vec
    # Ahora la condición inicial sobre el arreglo.
    e_vec[1] = exp(a.taylor_vec[1])
    # El siguiente for es la relación de recurrencia hallada que
    # usa los valores E_k previamente obtenidos.
    for k in range(2,size(e_vec,1)-1)
        e_vec[k] = (1/(k-1))*sum([(k-j)*e_vec[j]*a.taylor_vec[k-j+1] for j in range(1,k)])
    end
    # Se regresa la estructura de Taylor con coeficientes calculados.
    return Taylor(e_vec)
end

exp (generic function with 14 methods)

Probemos con $g(x) = (1+x)^2$:

In [10]:
a = Taylor([1,1])*Taylor([1,1])

Taylor{Int64}([1,2,1,0,0,0,0,0,0,0])

In [11]:
exp(a)

Taylor{Float64}([2.718281828459045,5.43656365691809,8.154845485377136,9.060939428196818,8.607892456786978,7.067532753993518,5.225141736926831,3.512192711691528,2.1843336121545898,1.2658947386324706])

Se ve muy claro que la derivada en $x=0$ es $2e$ como debería ser. La segunda derivada es según [Wolfram](http://www.wolframalpha.com/input/?i=Exp%28%281%2Bx%29^2%29+second+derivative+at+x+%3D0]) $6e\approx 16.31$. Nuestro coeficiente es la mitad, también como se espera.

Ahora que pudimos implementar la exponencial debemos implementar las otras funciones. Esto será muy sencillo una vez que obtengamos las relaciones de recurrencia. Para el logaritmo tenemos (puesta con índices que empiezan en 1 por convenienza):

$$L(x) = \log(g(x)) = \sum_{k=1}^\infty L_k (x-x_0))^{k-1}$$

$$\frac{dL(x)}{dx} = \sum_{k=1}^\infty k L_{k+1}(x-x_0)^{k-1} = \frac{1}{g(x)}\,\frac{dg(x)}{dx} = \frac{\sum_{j=1}^\infty j g_{j+1} (x-x_0)^{j-1}}{g(x)}$$

Por lo tanto se ve que para sacar los $L_k$ debemos dividir las series de Taylor de $g'$ y $g$, sacar el coeficiente de orden $k$ y dividir entre $k$ finalmente. La condición inicial es obviamente

$$L_1 = \log(g_1)$$

In [12]:
import Base: log
# Esta es la función logaritmo aplicada a estructuras Taylor.
function log(a::Taylor)
    # Este primer if está armado para que no se regrese un error de
    # inexactitud de meter un entero a un logaritmo que trabaja 
    # mínimo con números flotantes.
    if eltype(a.taylor_vec) <: Integer
        a = Taylor(convert(Array{Float64,1}, a.taylor_vec))
    end
    # Inicializamos el arreglo e_vec que será quien llene la 
    # estructura Taylor asociada al logaritmo:
    l_vec = 0.*a.taylor_vec
    # Ahora la condición inicial sobre el arreglo.
    l_vec[1] = log(a.taylor_vec[1])
    # Ahora vamos a crear el arreglo de la derivada de a.
    b = Taylor(convert(Array{eltype(l_vec),1},[i*a.taylor_vec[i+1] for i in range(1,size(a.taylor_vec,1)-1)]))
    # El siguiente for es la relación de recurrencia hallada que
    # es básicamente el de la división de dos series de Taylor.
    for k in range(1,size(l_vec,1)-1)
        l_vec[k+1] = (1/k)*(b/a).taylor_vec[k]
    end
    # Se regresa la estructura de Taylor con coeficientes calculados.
    return Taylor(l_vec)
end

log (generic function with 21 methods)

In [13]:
log(a)

Taylor{Float64}([0.0,2.0,-1.0,0.6666666666666666,-0.5,0.4,-0.3333333333333333,0.2857142857142857,-0.25,0.2222222222222222])

Wolfram nos dice que la serie es:


$$\log((1+x)^2) = 2x-x^2+2x^3/4-x^4/2 +2x^5/5 +\ldots$$

Así que vemos que está bien implementado todo.

Para completar el logaritmo y la exponencial, esperamos que sean inversas:

In [14]:
log(exp(a))

Taylor{Float64}([1.0,2.0,1.0000000000000002,0.0,0.0,0.0,-4.356567742664224e-16,3.734200922283621e-16,1.6337129034990842e-16,-2.904378495109483e-16])

Se ve que esto es cierto, ya que la serie tiene errores despreciables de orden $10^{-16}$.

In [15]:
exp(log(a))

Taylor{Float64}([1.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0])

Teniendo el logaritmo y la exponencial es muy sencillo implementar la potencia pues

$$P_a(x) = \exp(a\log(g(x))$$

$$\frac{dP_a(x)}{dx} = a(g(x))^{a-1} = a\,\exp((a-1)\,\log(g(x)))\cdot\frac{dg(x)}{dx}$$

In [16]:
import Base: ^
# Recuérdese que Julia pide primero declararla para exponente entero.
function ^(a::Taylor,b::Integer)
    c = Taylor(1)
    for i in range(1,b)
        c = c*a
    end
    return c
end
# La implementación no entera se hace con la exponencial-logaritmo.
^(a::Taylor,b::Number) = exp(b*log(a))

^ (generic function with 49 methods)

Así salen cosas muy evidentes como:

In [17]:
a^2

Taylor{Int64}([1,4,6,4,1,0,0,0,0,0])

Pero también cosas más sorprendetes:

In [18]:
a^(1/2)

Taylor{Float64}([1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0])

In [19]:
(a^(2im)).taylor_vec

10-element Array{Complex{Float64},1}:
      1.0+0.0im    
      0.0+4.0im    
     -8.0-2.0im    
      8.0-9.33333im
  3.33333+15.0im   
 -14.6667-9.33333im
  18.4444-2.0im    
 -14.6667+12.254im 
  6.70635-18.0556im
  2.06349+19.03im  

Nuevamente Wolfram nos dice que esta serie es 

$$1 + 4ix-(8+2i)x^2 + (8-28i/3)x^3 +\ldots$$

Para finalizar, queda claro que con la potencia podemos construir casi de manera inmediata el coseno y seno utilizando que 

$$\cos(g(x)) =\sum_{k=0}^\infty \frac{(-1)^{k}}{(2k)!}\,(g(x))^{2k}$$

$$\sin(g(x)) = \sum_{k=0}^\infty \frac{(-1)^{k}}{(2k+1)!}\,(g(x))^{2k+1}$$

In [20]:
import Base: sin, cos
# Teniendo la potencia a nuestra disposición es casi trivial escribir
# las funciones coseno y seno en términos de su serie de pontecias.
cos(a::Taylor) = sum([((-1)^(k)/(factorial(2*k)))*a^(2*k) for k in range(0,size(a.taylor_vec,1)-1)])
sin(a::Taylor) = sum([((-1)^(k)/(factorial(2*k+1)))*a^(2*k+1) for k in range(0,size(a.taylor_vec,1)-1)])

sin (generic function with 13 methods)

In [21]:
sin(Taylor([0]))

Taylor{Float64}([0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0])

In [22]:
cos(Taylor([0]))

Taylor{Float64}([1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0])

In [23]:
cos(a)

Taylor{Float64}([0.5403023058681398,-1.6829419696157875,-1.9220755965440781,0.04135670134202675,1.7729923539363124,1.3374817967419237,0.0715399421341259,-0.503589809840182,-0.35985058441647916,-0.06545891257386795])

In [24]:
sin(a)

Taylor{Float64}([0.8414709848078965,1.0806046117362795,-1.1426396637476477,-2.4033450441065765,-0.9403594476010254,0.7257396221113361,1.0368247168927456,0.4025776396788715,-0.10801246692651392,-0.1918756431681469])